<a href="https://colab.research.google.com/github/code-joker07/Healthcare-Chatbot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.6 MB/s eta 0:00:00


In [ ]:
import openai
import re

# Replace 'YOUR_OPENAI_API_KEY' with your actual API key
API_KEY = 'YOUR_OPENAI_API_KEY'
openai.api_key = API_KEY

class GuardedChatbot:
    HARMFUL_PATTERNS = [
        r"suicide",
        r"hurt myself",
        r"kill myself",
        r"self[\s-]?harm",
        r"self[\s-]?hate",
        r"overdose",
        r"end my life",
        r"can't go on",
        r"don't want to live",
        r"nobody would miss me",
        r"better off without me",
        r"feel worthless",
        r"no reason to live",
        r"want to die",
        r"shouldn't be alive",
        r"cut myself",
        r"drink bleach",
        r"swallow pills",
        r"jump off",
        r"hang myself",
        r"need a noose",
        r"pull the trigger",
        r"gun to my head",
        r"end it all",
        r"no point in living",
        r"give up on life",
        r"take my own life",
        r"can't bear it",
        r"want out",
        r"life isn't worth it",
        r"too much pain",
        r"nothing to live for",
        r"want everything to end",
        r"step in front of a train",
        r"need a way out",
        r"don't want to wake up",
        r"hope I don't wake up",
        r"wish I were dead",
        r"can't face another day",
        r"done with life",
        r"death is the only option",
        r"no hope left",
        r"can't see a way out",
        r"not worth saving",
        r"better off dead",
        r"can't find a reason",
        r"no future",
        r"tired of living",
        r"end the pain",
        # Add more patterns as needed
    ]

    EMERGENCY_PATTERNS = [
        r"emergency",
        r"urgent",
        r"hurry",
        r"immediate",
        r"help now",
        # Add more patterns as needed
    ]

    def __init__(self):
        self.context_history = []
        self.in_sensitive_mode = False

    def contains_harmful_pattern(self, text):
        for pattern in self.HARMFUL_PATTERNS:
            if re.search(pattern, text, re.IGNORECASE):
                return True
        return False

    def contains_emergency_pattern(self, text):
        for pattern in self.EMERGENCY_PATTERNS:
            if re.search(pattern, text, re.IGNORECASE):
                return True
        return False

    def get_response(self, prompt):
        if self.contains_emergency_pattern(prompt):
            return ("This sounds serious. Please seek immediate assistance from local resources, "
                    "helplines, or professionals. I am not equipped to help in emergencies.")

        if self.contains_harmful_pattern(prompt) or self.in_sensitive_mode:
            self.in_sensitive_mode = True
            prompt = "How can I comfort someone who says: " + prompt

        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            max_tokens=150,
            temperature=0.2
        )

        chatbot_response = response['choices'][0]['text'].strip()
        self.context_history.append(prompt)

        # Check if the bot-generated response contains harmful or emergency patterns and adjust accordingly
        if self.contains_harmful_pattern(chatbot_response):
            return ("I'm sorry, I'm here to support. "
                    "It's crucial to talk to a mental health professional or another trusted expert for guidance.")

        if self.contains_emergency_pattern(chatbot_response):
            return ("Please remember to seek immediate assistance for any emergencies. "
                    "I am a chatbot and cannot provide real-time support.")

        return chatbot_response

    def fallback_response(self):
        return "I'm truly here to help and support you. Please share your feelings or thoughts."

    def chat(self):
        print("Chatbot: Hi! How can I support you today? Please remember that I'm not a replacement for professional help.")
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                break

            chatbot_response = self.get_response(user_input)

            if chatbot_response == self.context_history[-1]:
                chatbot_response = self.fallback_response()

            print("Chatbot:", chatbot_response)

if __name__ == '__main__':
    chatbot = GuardedChatbot()
    chatbot.chat()